In [2]:
import getpass
import os

# Set LangSmith Environment Variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith")


In [3]:
# Set up google llm model for RAG usage
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


print("Initializing chat model!")
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
print("Chat model initialization over!")

Initializing chat model!
Chat model initialization over!


In [4]:
print("Initializing embedder model!")
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print("Embedder model initialized!")

Initializing embedder model!
Embedder model initialized!


In [5]:
print("Initialize vector store!")
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
print("Vector store initialized!")

Initialize vector store!
Vector store initialized!


In [7]:
# Loading documents for RAG use

# get beautiful soup
import bs4
# get Document Loaders
from langchain_community.document_loaders import WebBaseLoader
desired_webpage_link = "https://lilianweng.github.io/posts/2023-06-23-agent/"
# the original tutorial only wanted us to keep html tags that
# contain any of the class names listed in the tuple assigned to the 
# class_ keyword, in the below variable. We discard all other html tags.
bs4_filtered = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

webpage_loader = WebBaseLoader(
  web_paths=(desired_webpage_link,),
  bs_kwargs={"parse_only": bs4_filtered}
)

documents = webpage_loader.load()
#for key in documents[0]:
  #print(f"New key: {key}")
#print(f"how many characters are in our page: {len(documents[0].page_content)}")
#print(f"The first two thousand characters of the document: {documents[0].page_content[:2000]}")

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# create a text splitter instance
recursive_text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200,
  add_start_index=True,
)

# obtain splitted text. our document is now a bunch of document shards.
splitted_text = recursive_text_splitter.split_documents(documents)
#print(f"The splits we got back:\n\n{splitted_text[4].page_content}")
#print(f"Number of splits:{len(splitted_text)}")

# We've loaded in our document. 
Split it up into text chunks, to allow our model to access bits of the data that fit into the LLM context window.

Now, it's time to put these chunks into a storage format the LLM can access.

In [10]:
# the vector store was previously initialized with our defined embeddings
# in one command sequence, we tell the vector store to embed each of our document chunks.
# we then store the embedded versions of these chunks into our vector store, for 
# later retrieval
vector_store__document_embedded_vectors = vector_store.add_documents(splitted_text)


In [24]:
print(f"vector store shape: {vector_store__document_embedded_vectors}")

vector store shape: ['9e7d4870-5720-4526-8f77-e36463bc63e2', 'a641e2ab-fb0d-4f70-b70c-23e6806197cf', '4bec83f9-0f77-4b2c-b65c-e8204629e5ef', 'cdfecf70-e108-4e00-847a-13555c6eac38', '5fa4ec99-0bdc-45fa-b64a-60c796255246', '88d537ed-3ee8-4cda-a1ee-da2a7652c6ab', '3d74db9d-dba9-4c7c-8bb2-dd8ad8915b79', '267b6707-11f3-4b4c-bcd4-da91d5764c00', '13528c3f-e17a-4582-bdab-d8ae4c4926f2', '6cb5d11a-550a-409b-baad-664faf669cda', 'e04f327b-b3ca-4511-bac2-9b5ce05db924', '389bba3a-0a1d-4456-8b68-7176e10d80e1', '82be8ba8-50b6-4838-8bfd-f556c37acacb', 'ac133bdc-072c-4489-bc4b-1693938ae216', '94ee69d2-3acf-44ee-97d6-4e1f4cc6e17b', '5fa6a103-8913-43be-9076-d0808a74f1e3', 'dc9af879-20d1-4335-87e7-ceafe0f6ca13', 'd7a4a3c5-1f18-45c8-b43c-bbb34ad634e5', 'ba0adb1e-7bfd-46f2-926b-78a1fdd34454', 'c6330ffa-4f71-494d-9e44-ed1d037f77f7', '6d431b74-77f0-453f-a43c-5f489eeb9134', '98599d29-aee2-45ed-9e91-196d2dc9f43b', 'a0d640f2-9073-4423-a9e0-482604af3ac8', '290eae8d-c363-4e77-828f-77eb569339b5', '2fa6b39a-2289-439a

# We now should be able to answer User Queries
By accessing our vector store and returning the relevant chunks of text, from Lillian's blog post, that answers the user's questions.